In [8]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from transformers import AutoTokenizer, AutoModel, ViTFeatureExtractor, ViTModel
from PIL import Image
import joblib
import os
from tqdm import tqdm
from sklearn.model_selection import train_test_split

# Define equipment
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Example DataFrame (please adjust according to your data source)
# df = pd.read_csv('your_data.csv')  # Load DataFrame based on your data source
# Read CSV file
df = pd.read_csv("Data.csv", encoding='utf-8', encoding_errors='ignore')

# Rename the column (ensure consistency with the previous one)
df.columns = [
    'ProductName', 'PrimaryCategory', 'SecondaryCategory', 'url', 'sku', 'TextDescription', 'Discount',
    'DiscountedPrice', 'Price', 'StarRating', 'Sales', 'Revenue', 'CleanedReviews', 'cc-1', 'cc-2',
    'cc-3', 'DRC-1', 'DRC-2', 'DRC-3', 'RCV-1', 'RCV-2',
    'RCV-3', 'RSV-1', 'RSV-2', 'RSV-3'
]

# Convert data type
numeric_columns = ['StarRating', 'Discount', 'DiscountedPrice', 'Price', 'Sales', 'Revenue']
for col in numeric_columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Recalculate income (if necessary)
df['Revenue'] = df['DiscountedPrice'] * df['Sales']
df['Revenue'] = np.log10(df['Revenue'])

# Numerical characteristics
numerical_features = ['DiscountedPrice', 'Discount', 'Price']

# Standardized numerical features
scaler = StandardScaler()
df[numerical_features] = scaler.fit_transform(df[numerical_features])

# Save the Scaler for future use
joblib.dump(scaler, 'scaler.pkl')


# Set up device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Initialize BERT model and tokenizer
bert_model_name = r"/root/.cache/huggingface/hub/models--google-bert--bert-base-uncased/snapshots/86b5e0934494bd15c9632b12f734a8a67f723594"
tokenizer_bert = AutoTokenizer.from_pretrained(bert_model_name)
model_bert = AutoModel.from_pretrained(bert_model_name, from_tf=False)  # Set according to the actual situation
model_bert.to(device)
model_bert.eval()  # Set to evaluation mode

def extract_text_features(texts, model, tokenizer, device, batch_size=32, max_length=128):
    all_features = []
    num_samples = len(texts)
    num_batches = int(np.ceil(num_samples / batch_size))
    
    with torch.no_grad():  # Disable gradient computation
        for batch_idx in tqdm(range(num_batches), desc="Extract BERT text features"):
            start_idx = batch_idx * batch_size
            end_idx = min(start_idx + batch_size, num_samples)
            batch_texts = texts[start_idx:end_idx]
            
            # Tokenize with fixed max_length and padding to max_length
            inputs = tokenizer(
                batch_texts,
                return_tensors='pt',
                padding='max_length',  # Fixed padding to max_length
                truncation=True,
                max_length=max_length,
            )
            inputs = {key: val.to(device) for key, val in inputs.items()}
            
            # Obtain model output
            outputs = model(**inputs)
            
            # Perform average pooling on the last hidden state to obtain fixed-dimensional features for each sample
            encoded_text = outputs.last_hidden_state.mean(dim=1).cpu().numpy()  # [batch_size, hidden_size]
            
            # Add to feature list
            all_features.append(encoded_text)
    
    # Stack the features of all batches to obtain [num_samples, hidden_size]
    all_features = np.vstack(all_features)
    return all_features



# Ensure that all text fields are of string type
text_columns = [ 'ProductName', 'PrimaryCategory', 'SecondaryCategory', 'TextDescription', 'cc-1']
for col in text_columns:
    df[col] = df[col].astype(str)

# Build review and basic text features
other_texts = (
    df['ProductName'] + " " +
    df['PrimaryCategory'] + " " +
    df['SecondaryCategory'] + " " +
    df['cc-1']
).tolist()

# Construct "text description" features
text_descs = df['TextDescription'].tolist()

# Extracting the "review and basic text encoding" features of BERT
bert_other_text_features = extract_text_features(
    other_texts,
    model_bert,
    tokenizer_bert,
    device,
    batch_size=32,
    max_length=128
)
df['other_text_features'] = list(bert_other_text_features)

# Extract the "text description" feature of BERT
bert_text_desc_features = extract_text_features(
    text_descs,
    model_bert,
    tokenizer_bert,
    device,
    batch_size=32,
    max_length=128
)
df['text_desc_features'] = list(bert_text_desc_features)

print("BERT text feature extraction is complete.")


Extract BERT text features: 100%|██████████| 7/7 [00:00<00:00, 39.56it/s]

BERT text feature extraction is complete.


In [9]:
import torchvision.models as models
import torchvision
from torchvision import transforms
# Image feature extraction
def vit_encoding(image_dir, num_images=2194, device='cpu'):
    vit_list = []
    model_name = "/root/.cache/huggingface/hub/models--google--vit-base-patch16-224/snapshots/3f49326eb077187dfe1c2a2bb15fbd74e6ab91e3"
    feature_extractor = ViTFeatureExtractor.from_pretrained(model_name)
    model = ViTModel.from_pretrained(model_name).to(device).eval()

    def process_image(image_path):
        try:
            image = Image.open(image_path).convert('RGB')
            return feature_extractor(images=image, return_tensors="pt")
        except Exception as e:
            print(f"无法打开图片 {image_path}: {e}")
            # Return a tensor of all zeros to maintain consistent dimensions
            return feature_extractor(images=Image.new('RGB', (224, 224), (0, 0, 0)), return_tensors="pt")

    for i in tqdm(range(num_images), desc="Extract image features"):
        image_path = os.path.join(image_dir, f"{i}.jpg")
        inputs = process_image(image_path)
        inputs = {k: v.to(device) for k, v in inputs.items()}
        with torch.no_grad():
            outputs = model(**inputs)
            cls_token = outputs.last_hidden_state[:, 0, :].cpu().numpy()  # [1, hidden_size]
            vit_list.append(cls_token.squeeze())
    
    vit_np = np.array(vit_list)  # [num_images, hidden_size]
    
    # PCA dimensionality reduction
    vit_pca = PCA(n_components=50)
    vit_features_pca = vit_pca.fit_transform(vit_np)
    
    joblib.dump(vit_pca, 'vit_pca.pkl')
    
    return vit_features_pca

# Define using your image path
image_directory = r"/root"
vit_features = vit_encoding(image_directory, num_images=len(df))
df['vit_features'] = list(vit_features)


/root/miniconda3/lib/python3.12/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(
Some weights of ViTModel were not initialized from the model checkpoint at /root/.cache/huggingface/hub/models--google--vit-base-patch16-224/snapshots/3f49326eb077187dfe1c2a2bb15fbd74e6ab91e3 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
提取图像特征: 100%|██████████| 202/202 [00:09<00:00, 21.11it/s]


In [10]:
# PSI framework
class RegressionModelWithAttention(nn.Module):
    def __init__(self, input_dim_image, input_dim_other_text, input_dim_text_desc, input_dim_numerical, attention_dim=128):
        super(RegressionModelWithAttention, self).__init__()
        # Image feature processing
        self.image_dense = nn.Sequential(
            nn.Linear(input_dim_image, 256),
            nn.Sigmoid(),
            nn.Dropout(0.5)
        )
        # Other text feature processing
        self.other_text_dense = nn.Sequential(
            nn.Linear(input_dim_other_text, 256),
            nn.Sigmoid(),
            nn.Dropout(0.5)
        )
        # "Text description" feature processing (removing the self-attention layer)
        self.text_dense = nn.Sequential(
            nn.Linear(input_dim_text_desc, 256),
            nn.ReLU(),
            nn.Dropout(0.5)
        )
        # Numerical feature processing
        self.numerical_dense = nn.Sequential(
            nn.Linear(input_dim_numerical, 64),
            nn.Sigmoid(),
            nn.Dropout(0.3)
        )
        # Fusion featuresArithmeticError
        self.fusion = nn.Sequential(
            nn.Linear(256 + 256 , 256),  # Image features + textual descriptive features
            nn.Sigmoid(),
            nn.Dropout(0.5)
        )
        self.combined = nn.Sequential(
            nn.Linear(256 + 256 + 64, 128),  # Other text features + fused features + numerical features
            nn.Sigmoid(),
            nn.Dropout(0.3)
        )
        # Output layer
        self.output = nn.Linear(128, 1)  # Regression output

    def forward(self, image, other_text, text_desc, numerical):
        # Image feature processing
        image_features = self.image_dense(image)  # [batch_size, 256]
        # Other text feature processing
        other_text_features = self.other_text_dense(other_text)  # [batch_size, 256]
        # "Text description" feature processing (directly passed to the fully connected layer)
        text_desc_features = self.text_dense(text_desc)  # [batch_size, 256]
        # Numerical feature processing
        numerical_features = self.numerical_dense(numerical)  # [batch_size, 64]
        # Integrate image, other text, and textual descriptive features
        fused = torch.cat((image_features, text_desc_features), dim=1)  # [batch_size, 768]
        fused = self.fusion(fused)  # [batch_size, 256]
        # Concatenation with numerical features
        combined = torch.cat((fused, numerical_features,other_text_features), dim=1)  # [batch_size, 320]
        combined = self.combined(combined)  # [batch_size, 128]
        # output
        output = self.output(combined)  # [batch_size, 1]
        return output


In [11]:
# Define the Dataset class
class IncomeDataset(Dataset):
    def __init__(self, df, image_feature_type='vit'):
        """
        Initialize the dataset.
        Parameters:
            df (pd.DataFrame): Data frame.
        """
        supported_features = ['vit_features']
        feature_mapping = {
            'vit': 'vit_features',
        }
        if image_feature_type not in feature_mapping:
            raise ValueError(f"Unsupported image feature type: {image_feature_type}. Supported types: {list(feature_mapping.keys())}")
        selected_feature = feature_mapping[image_feature_type]
        self.image_features = np.stack(df[selected_feature].values)
        self.other_text_features = np.stack(df['other_text_features'].values)
        self.text_desc_features = np.stack(df['text_desc_features'].values)
        self.numerical_features = df[[ 'Discount','DiscountedPrice', 'Price']].values
        self.targets = df['Revenue'].values.astype(np.float32)
        self.image_feature_type = image_feature_type

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, idx):
        image = self.image_features[idx]
        other_text = self.other_text_features[idx]
        text_desc = self.text_desc_features[idx]
        numerical = self.numerical_features[idx]
        target = self.targets[idx]
        return {
            'image': torch.tensor(image, dtype=torch.float32),
            'other_text': torch.tensor(other_text, dtype=torch.float32),
            'text_desc': torch.tensor(text_desc, dtype=torch.float32),
            'numerical': torch.tensor(numerical, dtype=torch.float32),
            'target': torch.tensor(target, dtype=torch.float32)
        }


In [12]:
# Define training and evaluation functions
def train_and_evaluate(model, train_loader, val_loader, criterion, optimizer, device, num_epochs=100):
    """
    Train and evaluate the model.

    Parameters:
        model (nn.Module): The model to be trained.
        train_loader (DataLoader): Training data loader.
        val_loader (DataLoader): Validation data loader.
        criterion: loss function.
        optimizer: optimizer.
        device (str): Device type.
        num_epochs (int): Number of training epochs.
    """
    train_losses = []
    val_losses = []
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training"):
            optimizer.zero_grad()
            outputs = model(
                batch['image'].to(device),
                batch['other_text'].to(device),
                batch['text_desc'].to(device),
                batch['numerical'].to(device)
            ).squeeze()
            loss = criterion(outputs, batch['target'].to(device))
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * batch['image'].size(0)
        train_loss /= len(train_loader.dataset)
        train_losses.append(train_loss)

        # validation phase
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for batch in tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Validation"):
                outputs = model(
                    batch['image'].to(device),
                    batch['other_text'].to(device),
                    batch['text_desc'].to(device),
                    batch['numerical'].to(device)
                ).squeeze()
                loss = criterion(outputs, batch['target'].to(device))
                val_loss += loss.item() * batch['image'].size(0)
        val_loss /= len(val_loader.dataset)
        val_losses.append(val_loss)

        print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {train_loss:.4f} - Val Loss: {val_loss:.4f}")

    # Evaluate after all epochs have ended
    all_predictions_log, all_targets_log, all_targets, all_predictions = evaluate_model(model, val_loader, device)
    return all_predictions_log, all_targets_log, all_targets, all_predictions

In [13]:
# Set random seed
import random
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from transformers import set_seed
SEEDS = [42]


# Define different types of image features
image_feature_types = ['vit']  # 假设 'resnet_features' 已提取



# Define model parameters (adjusted according to the dimensions of different features)
FEATURE_DIMENSIONS = {
    'vit': 50,          # ViT feature dimension after PCA

}

input_dim_other_text = 768  # The hidden_size of RoBERTa
input_dim_text_desc = 768   # Adjust according to the actual hidden_size
input_dim_numerical = 3     # Number of numerical features

# Define training parameters
batch_size = 32
num_epochs = 1000

# Define loss function
criterion = nn.MSELoss()

# inspect equipment
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Traverse all random seeds
for seed in SEEDS:
    print(f"\n=== 使用随机种子 {seed} 进行数据划分 ===")
    set_seed(seed)

    # Dataset partitioning
    train_df, val_df = train_test_split(
        df,
        test_size=0.01,  # Adjust the proportion of the validation set as needed
        random_state=seed,
        shuffle=True
    )

    # Traverse all image feature types
    for feature_type in image_feature_types:
        print(f"\n=== 训练使用 {feature_type} 特征的模型 ===")

        # Define input dimensions
        input_dim_image = FEATURE_DIMENSIONS.get(feature_type)
        if input_dim_image is None:
            print(f"未定义特征类型 {feature_type} 的输入维度。跳过...")
            continue

        # Create training set and validation set
        train_dataset = IncomeDataset(train_df, image_feature_type=feature_type)
        val_dataset = IncomeDataset(val_df, image_feature_type=feature_type)

        # Create a data loader
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

        # Instantiation model
        model = RegressionModelWithAttention(
            input_dim_image=input_dim_image,
            input_dim_other_text=input_dim_other_text,
            input_dim_text_desc=input_dim_text_desc,
            input_dim_numerical=input_dim_numerical,
            attention_dim=128
        ).to(device)

        # Define optimizer
        optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

        # Train and evaluate the model
        all_predictions_log, all_targets_log, all_targets, all_predictions = train_and_evaluate(
            model,
            train_loader,
            val_loader,
            criterion,
            optimizer,
            device,
            num_epochs=num_epochs
        )
    # Convert four lists into a dictionary
        data = {
    'all_predictions_log': all_predictions_log,
    'all_targets_log': all_targets_log,
    'all_predictions': all_predictions,
    'all_targets': all_targets
        }
        # Convert the dictionary into a pandas DataFrame
        df1 = pd.DataFrame(data)
        # Save to a CSV file, specifying the file name (e.g. 'predictions_targets.csv')
        df1.to_csv('predictions_targets'+str(seed)+'.csv', index=False)

        # Save the model (optional)
        model_save_path = f"regression_model_{feature_type}_seed{seed}.pth"
        torch.save(model.state_dict(), model_save_path)


Using device: cuda

=== 使用随机种子 42 进行数据划分 ===

=== 训练使用 vit 特征的模型 ===


Epoch 1/1000 - Validation: 100%|██████████| 1/1 [00:00<00:00, 791.08it/s]


Epoch 1/1000 - Train Loss: 3.2553 - Val Loss: 0.2176


Epoch 2/1000 - Validation: 100%|██████████| 1/1 [00:00<00:00, 830.56it/s]


Epoch 2/1000 - Train Loss: 0.4195 - Val Loss: 0.1305


...

Epoch 999/1000 - Validation: 100%|██████████| 1/1 [00:00<00:00, 861.25it/s]


Epoch 999/1000 - Train Loss: 0.0220 - Val Loss: 0.0405


Epoch 1000/1000 - Validation: 100%|██████████| 1/1 [00:00<00:00, 774.86it/s]


Epoch 1000/1000 - Train Loss: 0.0254 - Val Loss: 0.0632


NameError: name 'evaluate_model' is not defined